# Agent Reasoning Loop

#### This notebook is a reproduction of the experiment3 presented in the Building Agentic RAG with Llamaindex Course
#### Source: https://learn.deeplearning.ai/courses/building-agentic-rag-with-llamaindex/lesson/4/building-an-agent-reasoning-loop

## Setup

In [1]:
from helper import get_openai_api_key
OPENAI_API_KEY = get_openai_api_key()

In [2]:
import nest_asyncio
nest_asyncio.apply()

## Load the data

In [3]:
!wget "https://openreview.net/pdf?id=VtmBAGCN7o"

--2024-05-10 00:39:19--  https://openreview.net/pdf?id=VtmBAGCN7o
Resolvendo openreview.net (openreview.net)... 35.184.86.251
Conectando-se a openreview.net (openreview.net)|35.184.86.251|:443... conectado.
A requisição HTTP foi enviada, aguardando resposta... 200 OK
Tamanho: 16911937 (16M) [application/pdf]
Salvando em: “pdf?id=VtmBAGCN7o.2”

pdf?id=VtmBAGCN7o.2 100%[===================>]  16,13M  9,60MB/s    em 1,7s    

2024-05-10 00:39:22 (9,60 MB/s) - “pdf?id=VtmBAGCN7o.2” salvo [16911937/16911937]



## Setup the Query Tools

In [4]:
from utils import get_doc_tools

vector_tool, summary_tool = get_doc_tools("metagpt.pdf", "metagpt")

## Setup Function Calling Agent

In [5]:
from llama_index.llms.openai import OpenAI

llm = OpenAI(model="gpt-3.5-turbo", temperature=0)

In [6]:
from llama_index.core.agent import FunctionCallingAgentWorker
from llama_index.core.agent import AgentRunner

agent_worker = FunctionCallingAgentWorker.from_tools(
    [vector_tool, summary_tool], 
    llm=llm, 
    verbose=True
)
agent = AgentRunner(agent_worker)

In [7]:
response = agent.query(
    "Tell me about the agent roles in MetaGPT, "
    "and then how they communicate with each other."
)

Added user message to memory: Tell me about the agent roles in MetaGPT, and then how they communicate with each other.
=== Calling Function ===
Calling function: summary_tool_metagpt with args: {"input": "agent roles in MetaGPT"}
=== Function Output ===
The agent roles in MetaGPT include Product Manager, Architect, Project Manager, Engineer, and QA Engineer. Each role has specific responsibilities tailored to their expertise within the collaborative framework. The Product Manager conducts business-oriented analysis, the Architect translates requirements into system design components, the Project Manager handles task distribution, the Engineer executes code, and the QA Engineer formulates test cases to ensure code quality. These roles work together in a sequential manner following Standard Operating Procedures (SOPs) to efficiently tackle complex tasks within the MetaGPT framework.
=== Calling Function ===
Calling function: summary_tool_metagpt with args: {"input": "communication betwee

In [8]:
print(response.source_nodes[0].get_content(metadata_mode="all"))

page_label: 1
file_name: metagpt.pdf
file_path: metagpt.pdf
file_type: application/pdf
file_size: 16911937
creation_date: 2024-05-09
last_modified_date: 2024-05-09

Preprint
METAGPT: M ETA PROGRAMMING FOR A
MULTI -AGENT COLLABORATIVE FRAMEWORK
Sirui Hong1∗, Mingchen Zhuge2∗, Jonathan Chen1, Xiawu Zheng3, Yuheng Cheng4,
Ceyao Zhang4,Jinlin Wang1,Zili Wang ,Steven Ka Shing Yau5,Zijuan Lin4,
Liyang Zhou6,Chenyu Ran1,Lingfeng Xiao1,7,Chenglin Wu1†,J¨urgen Schmidhuber2,8
1DeepWisdom,2AI Initiative, King Abdullah University of Science and Technology,
3Xiamen University,4The Chinese University of Hong Kong, Shenzhen,
5Nanjing University,6University of Pennsylvania,
7University of California, Berkeley,8The Swiss AI Lab IDSIA/USI/SUPSI
ABSTRACT
Remarkable progress has been made on automated problem solving through so-
cieties of agents based on large language models (LLMs). Existing LLM-based
multi-agent systems can already solve simple dialogue tasks. Solutions to more
complex tasks, however, 

In [9]:
response = agent.chat(
    "Tell me about the evaluation datasets used."
)

Added user message to memory: Tell me about the evaluation datasets used.
=== Calling Function ===
Calling function: summary_tool_metagpt with args: {"input": "evaluation datasets used in MetaGPT"}
=== Function Output ===
The evaluation datasets used in MetaGPT include HumanEval, MBPP, SoftwareDev, and humaneval-x.
=== LLM Response ===
The evaluation datasets used in MetaGPT include HumanEval, MBPP, SoftwareDev, and humaneval-x.


In [10]:
response = agent.chat("Tell me the results over one of the above datasets.")

Added user message to memory: Tell me the results over one of the above datasets.
=== Calling Function ===
Calling function: vector_tool_metagpt with args: {"query": "results over HumanEval dataset", "page_numbers": ["7"]}
=== Function Output ===
MetaGPT achieved 85.9% and 87.7% Pass rates over the HumanEval dataset.
=== LLM Response ===
MetaGPT achieved 85.9% and 87.7% Pass rates over the HumanEval dataset.


## Lower-Level: Debuggability and Control

In [12]:
agent_worker = FunctionCallingAgentWorker.from_tools(
    [vector_tool, summary_tool], 
    llm=llm, 
    verbose=True
)
agent = AgentRunner(agent_worker)

In [13]:
task = agent.create_task(
    "Tell me about the agent roles in MetaGPT, "
    "and then how they communicate with each other."
)

In [14]:
step_output = agent.run_step(task.task_id)


Added user message to memory: Tell me about the agent roles in MetaGPT, and then how they communicate with each other.
=== Calling Function ===
Calling function: summary_tool_metagpt with args: {"input": "agent roles in MetaGPT"}
=== Function Output ===
Agent roles in MetaGPT include the Product Manager, Architect, Project Manager, Engineer, and QA Engineer. The Product Manager is responsible for generating the Product Requirement Document (PRD) outlining goals, user stories, competitive analysis, and requirements. The Architect translates these requirements into technical specifications and system architecture. The Project Manager handles task distribution among the team members. Engineers are responsible for developing the code based on the assigned tasks. Lastly, the QA Engineer formulates unit test code and ensures the quality of the software solution.


In [15]:
completed_steps = agent.get_completed_steps(task.task_id)
print(f"Num completed for task {task.task_id}: {len(completed_steps)}")
print(completed_steps[0].output.sources[0].raw_output)

Num completed for task 712384e6-65fd-4ab8-9143-1527642627d5: 1
Agent roles in MetaGPT include the Product Manager, Architect, Project Manager, Engineer, and QA Engineer. The Product Manager is responsible for generating the Product Requirement Document (PRD) outlining goals, user stories, competitive analysis, and requirements. The Architect translates these requirements into technical specifications and system architecture. The Project Manager handles task distribution among the team members. Engineers are responsible for developing the code based on the assigned tasks. Lastly, the QA Engineer formulates unit test code and ensures the quality of the software solution.


In [16]:
upcoming_steps = agent.get_upcoming_steps(task.task_id)
print(f"Num upcoming steps for task {task.task_id}: {len(upcoming_steps)}")
upcoming_steps[0]

Num upcoming steps for task 712384e6-65fd-4ab8-9143-1527642627d5: 1


TaskStep(task_id='712384e6-65fd-4ab8-9143-1527642627d5', step_id='35d36b69-d0a0-4cf6-82f9-8e6499726997', input=None, step_state={}, next_steps={}, prev_steps={}, is_ready=True)

In [17]:
step_output = agent.run_step(
    task.task_id, input="What about how agents share information?"
)

Added user message to memory: What about how agents share information?
=== Calling Function ===
Calling function: summary_tool_metagpt with args: {"input": "how agents share information in MetaGPT"}
=== Function Output ===
Agents in MetaGPT share information through a structured communication protocol that includes a shared message pool where agents publish structured messages. They can also subscribe to relevant messages based on their profiles. This approach enhances role communication efficiency by allowing agents to obtain directional information from other roles and public information from the environment. Additionally, MetaGPT implements a publish-subscribe mechanism that enables agents to exchange information effectively within the multi-agent system. Agents also review previous feedback before each project, make adjustments to their constraint prompts based on this feedback, and utilize a handover feedback action to summarize and integrate information received during the develo

In [18]:
step_output = agent.run_step(task.task_id)
print(step_output.is_last)

=== LLM Response ===
Agents in MetaGPT share information through a structured communication protocol that includes a shared message pool where agents publish structured messages. They can also subscribe to relevant messages based on their profiles. This approach enhances role communication efficiency by allowing agents to obtain directional information from other roles and public information from the environment. Additionally, MetaGPT implements a publish-subscribe mechanism that enables agents to exchange information effectively within the multi-agent system. Agents also review previous feedback before each project, make adjustments to their constraint prompts based on this feedback, and utilize a handover feedback action to summarize and integrate information received during the development of previous projects into an updated constraint prompt stored in long-term memory for future use.
True


In [19]:
response = agent.finalize_response(task.task_id)

In [20]:
print(str(response))

assistant: Agents in MetaGPT share information through a structured communication protocol that includes a shared message pool where agents publish structured messages. They can also subscribe to relevant messages based on their profiles. This approach enhances role communication efficiency by allowing agents to obtain directional information from other roles and public information from the environment. Additionally, MetaGPT implements a publish-subscribe mechanism that enables agents to exchange information effectively within the multi-agent system. Agents also review previous feedback before each project, make adjustments to their constraint prompts based on this feedback, and utilize a handover feedback action to summarize and integrate information received during the development of previous projects into an updated constraint prompt stored in long-term memory for future use.
